In [1]:
import numpy as np
import pandas as pd
import datetime

# 数据来源

1. ERP查询的两天信息
    * 考虑到昨天下单、昨天=天发货的要求订单较少，先找今天发货的订单
2. 【\\192.168.1.81\物流部\专版发货资料\5月】的专版 当天的，导出当月的

In [13]:
data = pd.read_excel('5月专版发货明细【取自新专版返货明细22-23】.xlsx')
data.head(1)

,地区,客户名称,电话,地址,件数,发货日期,总款,定金,余额,交货方式,物流票号,物流单位,印刷类型,订单单号,专版理货员,列1
0,长治,捷曼广告,13453575764,长治,2,43590,560.0,0.0,560.0,专线运输,NaN,NaN,骑马钉画册,200258 /16618152,1,NaN


In [5]:
# 读取erp查询的专版订单数据
df_zhuanban = pd.read_excel('专版订单查询-190523.xls')
df_zhuanban.head(1)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,销售单号/订单编号,工单号,客户简称,销售客服,产品类别,下单日期,审核日期,交货日期,状态,订单数量,...,收款方式,收款状态,合同号,印件名称,打样状态,PO号,品名,出货状况,出货区域,确认电子书
0,207060/16824316,19052200377,金雅禾包装,范肖,折页,05-22 20:11,05-22 20:14,2019/5/23,生产中,25000.0,...,扣预存,未收款,E-0000,207060_西五 加急夏令营风琴六折页2种.zip,参数码样,NaN,NaN,NaN,西一线,无需确认


In [6]:
df_zhuanban['出货区域'].unique()

array(['西一线', '东一线', '运城市', '偃师', '北六', '临汾', '新厂自取', '新密市', '西五线', '北二线',
       '西二线', '北区开元印务', '安阳东区', '宝丰', '南三线', '北五线', '洛阳一区', '西安中区', '信阳市',
       '长垣', '北三线', '石家庄市', '固始', '栾川', '沁阳', '南一线', '周口一区', '鄢陵', '商丘市',
       '西峡', '许昌市', '北一线', '磁县', '巩义', '东二线', '扶沟', '北七线', '西三线', '内乡',
       '焦作市', '龙湖', '新郑市', '民权', '荥阳', '郸城', '南阳市', '濮阳市', '伊川', '晋城市',
       '新东线', '侯马', '汤阴', '安阳开发区', '汝州', '项城', '南二线', '顺丰快递', '修武', '商水县',
       '东三线', '开封一区', '济宁', '三门峡市', '镇平', '鲁山', '新安县', '洛阳三区', '开封市',
       '辉县', '新西线', '安阳北区', '鹿邑市', '尉氏', '安阳新区', '安阳西区', '石家庄', '原阳',
       '孟州', '唐河', '邯郸二区', '漯河市', '菏泽', '太康', '正阳县\r\n', '新蔡', '沙河市',
       '中牟', '邯郸三区', '温县', '平舆', '西四线', '港区', '洛阳二区', '汝阳', '济源', '延津',
       '嵩县', '豫鑫物流', '永年县', '开封三区', '沈丘', '春光物流', '西华县', '虞城', '蚌埠市',
       '驻马店市', '肥城', '封丘', '邓州', 'A区', '平顶山市', '邢台市', '武陟', '新区', nan],
      dtype=object)

In [7]:
#  郑州的配送站
areas_north = {'北一线':'北区','北二线':'北区','北三线':'北区','北四线':'北区','北五线':'北区','北六':'北区','北七线':'北区','北区印划算':'北区','北区中美':'北区','北区开源正彩':'北区','北区开元印务':'北区'}
areas_south = {'南一线':'南区','南二线':'南区','南三线':'南区','南区太阳印务':'南区','港区':'南区','龙湖':'南区'}
areas_east = {'东一线':'东区','东二线':'东区','东三线':'东区','东区E399':'东区'}
areas_west = {'西一线':'西区','西二线':'西区','西三线':'西区','西四线':'西区','西五线':'西区'}

areas ={}
areas.update(areas_east)
areas.update(areas_south)
areas.update(areas_north)
areas.update(areas_west)

* 配送区域 的划分
    * 物流、新厂自取、郑州区的，排除
    * 这些物流是固定的吗？

In [8]:
# 添加大区字段
df_zhuanban['大区'] = df_zhuanban['出货区域'].map(areas)

# 排除郑州配送站，和 【新厂自取、顺丰快递、春光物流、豫鑫物流】的配送订单
df_zhuanban_final = df_zhuanban.query('大区 != ["北区","西区","南区","东区"]').query("出货区域!=['新厂自取','顺丰快递','春光物流','豫鑫物流']")

df_zhuanban_final.drop_duplicates(inplace=True) # 去重——针对所有字段
df_zhuanban_final.dropna(subset=['销售单号/订单编号'],inplace=True) #  订单为空值得  去除
# # 选取时间
df_zhuanban_final['2019'] = '2019-'
df_zhuanban_final['下单时间'] = df_zhuanban_final['2019']+df_zhuanban_final['下单日期']
df_zhuanban_final['下单时间'] = df_zhuanban_final['下单时间'].str.split(' ',expand=True)[0]
df_zhuanban_final['交货日期'] = df_zhuanban_final['交货日期'].astype('datetime64[D]')

# 今天  str
b = np.datetime64('now').astype('datetime64[D]')
df1=df_zhuanban_final[(df_zhuanban_final['交货日期']== np.datetime64('now').astype('datetime64[D]'))] # 选取 查询当天的订单

a = (datetime.datetime.now()- datetime.timedelta(days=1)).strftime('%Y-%m-%d') # 昨天
df2 = df_zhuanban_final[(df_zhuanban_final['下单时间']==a)  &(df_zhuanban_final['交货日期']==a)]
df3 = pd.concat([df1,df2])

* 排除条件
    1. 新加了大区  字段
    2. 排除了 郑州站【】
    3. 排除了
        * 新厂自取
        * 顺丰快递
        * 春光物流
        * 豫鑫物流
    4. 排除了 销售单号/订单编号 为 空 的  订单
    5. 选取了
        * 下单日期是昨天  + 交货日期是 昨天 的订单
        * 交货日期 是今天 的订单
        
    
        
     

In [9]:
df3.head(2)

,销售单号/订单编号,工单号,客户简称,销售客服,产品类别,下单日期,审核日期,交货日期,状态,订单数量,...,印件名称,打样状态,PO号,品名,出货状况,出货区域,确认电子书,大区,2019,下单时间
2,207055/16824359,19052200381,新会数码5,张淑静,单页,05-22 20:04,05-22 20:26,2019-05-23,生产中,10000.0,...,207055_运城 家福鱼火锅宣传单页.zip,参数码样,NaN,NaN,NaN,运城市,无需确认,NaN,2019-,2019-05-22
4,207053/16823508,19052200335,豫CI0133鑫鑫文印,张淑静,封皮,05-22 20:01,05-22 20:02,2019-05-23,已落版,1000.0,...,207053_偃师 寇店镇党员积分奖励封皮.zip,参数码样,NaN,NaN,NaN,偃师,无需确认,NaN,2019-,2019-05-22


In [10]:
datetime.date.today()

datetime.date(2019, 5, 23)

In [11]:
# 选取 最终所需 字段
df3 = df3[['出货区域','客户简称','销售单号/订单编号','总金额','产品类别','出货状况','交货日期']]

# 重命名 字段名 与 【运营表】 统一
df3 = df3.rename(columns={'出货区域':'县区级区域','客户简称':'客户名称','销售单号/订单编号':'订单编号','总金额':'订单金额','出货状况':'出货状态'})
df3.to_excel('非郑州查询{}结果【专版】.xlsx'.format(b),encoding='gbk',index=None) # 注意 文件命名不能包含特殊字符

* 重命名了各 字段 【以《外埠五月运营表》】为准
* 并 保存在了 本地 

In [14]:
data.head(2)

,地区,客户名称,电话,地址,件数,发货日期,总款,定金,余额,交货方式,物流票号,物流单位,印刷类型,订单单号,专版理货员,列1
0,长治,捷曼广告,13453575764,长治,2,43590,560.0,0.0,560.0,专线运输,NaN,NaN,骑马钉画册,200258 /16618152,1,NaN
1,通许,恒基广告,13949429335,通许,2,43586,1060.0,0.0,1060.0,配送,NaN,NaN,定价档案袋,198456 /16563449,,NaN


In [15]:
#  截取ERP查询结果【订单编号】列中 的订单号——并去除 两端的空格
df3['erp查订单号'] = df3['订单编号'].str.split('/',expand=True)[0].str.strip()

# 同上，当天 【新专版发货明细22-23】 的 订单单号——并去除两端空格
data['专版发货明细订单号'] = data['订单单号'].str.split('/',expand=True)[0].str.strip()

* ERP查询单  与  共享盘 《新专版返货明细.xlsx》 按照 订单号 相关联 取出字段 并保存 在本地

In [16]:
# 将 两张 数据表 按照【订单号】 关联 ——由于是 outer 关联，所以
df3.merge(data,left_on='erp查订单号',right_on='专版发货明细订单号',how='outer')[['县区级区域','客户名称_x','订单编号','订单金额','产品类别','出货状态','交货日期','地区','件数','发货日期','交货方式','物流单位']].to_excel('【专版】非郑州最终结果{}.xlsx'.format(datetime.datetime.now().date().strftime('%Y-%m-%d')),encoding='gbk',index=None) # 注意 文件命名不能包含特殊字符